In [41]:
import cv2 
import matplotlib.pyplot as plot

from matplotlib import pyplot as plt
from matplotlib import image as image

import easygui
import numpy as np

import glob

In [42]:
#Face Folder
datasetSize = 'newx64\\'
faceDir = datasetSize + 'Faces\\'
notFaceDir = datasetSize + 'NotFaces\\'

In [43]:
#Create the HOG descriptor
winSize = (64,64)
blockSize = (16,16)
blockStride = (8,8)
cellSize = (8,8)
nbins = 9
derivAperture = 1
winSigma = -1.
histogramNormType = 0
L2HysThreshold = 0.2
gammaCorrection = 1
nlevels = 64
signedGradients = True

hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels, signedGradients)

In [44]:
#Get the images
faces = glob.glob(faceDir + '*.jpg')
nonFaces = glob.glob(notFaceDir + '*.jpg')
images = faces + nonFaces

#Create labels for the images
faceLabels = [1] * len(faces)
nonFaceLabels = [0] * len(nonFaces)
labels = faceLabels + nonFaceLabels

#Convert the lists into numpy arrays
images = np.array(images)
labels = np.array(labels)

#Shuffle the data
rand = np.random.RandomState(22)
shuffle = rand.permutation(len(images))
images, labels = images[shuffle], labels[shuffle]

print(np.unique(labels))
print(len(images))
print(len(faceLabels))

[0 1]
33187
7522


In [45]:
#Calculate HOG features for every image
descriptors = []

for img in images:
    img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    descriptor = hog.compute(img)
    descriptors.append(descriptor)

In [46]:
descriptors = np.squeeze(descriptors)

In [47]:
#Split the data into training and testing sets
#90% training - 10% testing

trainingSize = int(0.8 * len(descriptors))

imagesTrain, imagesTest = np.split(images, [trainingSize])
labelsTrain, labelsTest = np.split(labels, [trainingSize])
descriptorsTrain, descriptorsTest = np.split(descriptors, [trainingSize])

# #Double check
# print(len(descriptorsTrain))
# print(len(descriptorsTest))
# print(len(descriptors))
# print(labelsTrain)

In [48]:
#Initialise the SVM classifier
SVM = cv2.ml.SVM_create()
SVM.setKernel(cv2.ml.SVM_RBF)
SVM.setGamma(0.50625)
SVM.setType(cv2.ml.SVM_C_SVC)
SVM.setC(2.0)

#training
SVM.train(descriptorsTrain, cv2.ml.ROW_SAMPLE, labelsTrain)

True

In [ ]:
#Get the accuracy of the predictions on the test set
output = SVM.predict(descriptorsTest)[1].ravel()

accuracy = (labelsTest == output).mean()
print('Percentage Accuracy: %.2f %%' % (accuracy*100))

In [ ]:
#Get a confusion matrix for the test set
#ACTUAL VALUES       -Face- -NotFace- 
#PREDICTED -Face-    --TP-- --FP-- 
#PREDICTED -NotFace- --FN-- --TN-- 

confusion = np.zeros((2, 2), np.int32)
for i, j in zip(labelsTest, output):
    confusion[int(i), int(j)] += 1
print('confusion matrix:')
print(confusion)


In [ ]:
#Get all the images that were incorrectly labelled 
#https://stackoverflow.com/questions/46615554/how-to-display-multiple-images-in-one-figure-correctly/46616645
incorrectedLabeledImages = []
for i, j, k in zip(labelsTest, output, imagesTest):
    if i != j:
        incorrectedLabeledImages.append(k)


#Then 
w=10
h=10
fig=plt.figure(figsize=(8, 8))
columns = len(incorrectedLabeledImages) 
rows = 1
for i in range(1, columns*rows +1):
    img = cv2.imread(incorrectedLabeledImages[i-1])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()

In [ ]:
#SAVE THE MODEL
SVM.save("model.xml")
SVMextr = SVM

In [ ]:

model2 = cv2.ml.SVM_create()
model2 = model2.load("model.xml")

In [ ]:
#Get user to input file
f = easygui.fileopenbox()
img = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (64,64), interpolation = cv2.INTER_AREA)

plt.imshow(img, cmap='gray')

testDescriptor = hog.compute(img)
testDescriptor = np.array([testDescriptor])
prediction = model2.predict(testDescriptor)[1].ravel()
if prediction == 1:
    print("Face")
else:
    print("Not Face")

In [13]:
def slidingWindow(img):
    #Get the width and height
    markedImg = img.copy()
    grayImg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    height = img.shape[0]
    width = img.shape[1]

    #Define size of bounding box and step
    dim = 128
    step = 16
    
    for x in range(0, width, step):
        for y in range(0, height, step):
            if x+dim <= width and y+dim <= height:

                prediction = getPrediction(grayImg[y:y+dim, x:x+dim])
                if prediction == 1:
                    cv2.rectangle(markedImg, (x,y), (x+dim, y+dim), (0,255,0), 2)
                    print("detected")
   
    return markedImg

def getRandom(img):
    markedImg = img.copy()
    grayImg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    height = img.shape[0]
    width = img.shape[1]
    counter = 0
    
    xmin = np.random.randint(width, size=2000)
    xmax = np.random.randint(width, size=2000)
    ymin = np.random.randint(height, size=2000)
    ymax = np.random.randint(height, size=2000)
    
    for x1, x2, y1, y2 in zip(xmin, xmax, xmin, xmax):
        
        if y2 < y1:
            y2, y1 = y1, y2
        if x2 < x1:
            x2, x1 = x1, x2
        
        if x1 != x2 and y1 != y2:
            prediction = getPrediction(grayImg[y1:y2, x1:x2])
            if prediction == 1:
                cv2.rectangle(markedImg, (x1,y1), (x2, y2), (0,255,0), 2)
                print("detected")
                outputImg = img.copy()
                cv2.imwrite("outputs\\" + str(counter) + ".jpg", outputImg[y1:y2, x1:x2]) 
                counter += 1
        
    return markedImg
    
def getPrediction(img):
    try:
        img = cv2.resize(img, (64,64), interpolation = cv2.INTER_AREA)

        testDescriptor = hog.compute(img)
        testDescriptor = np.array([testDescriptor])
        prediction = SVM.predict(testDescriptor)[1].ravel()

        return prediction
    except:
        return 0


In [13]:
f = easygui.fileopenbox()
img = cv2.imread(f)
outputImg = slidingWindow(img)

cv2.imshow("img", outputImg)
cv2.waitKey(0)

AttributeError: 'NoneType' object has no attribute 'copy'

In [35]:
f = easygui.fileopenbox()
img = cv2.imread(f)
outputImg = getRandom(img)

cv2.imshow("img", outputImg)
cv2.waitKey(0)

-1

In [7]:
def getEllipses(img):
    #Get the width and height
    markedImg = img.copy()
    grayImg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    height = img.shape[0]
    width = img.shape[1]
    counter = 0
    
    faceCascade = cv2.CascadeClassifier('lbpcascade_animeface.xml')
    
    
    faces = faceCascade.detectMultiScale(grayImg,
                                 # detector options
                                 scaleFactor = 1.01,
                                 minNeighbors = 5,
                                 minSize = (32, 32))

    for (x, y, w, h) in faces:
        cv2.rectangle(markedImg, (x,y), (x + w, y + h), (0,0,255), 2)

        prediction = 0
        scale = 1.2
        cx = x + w/2
        cy = y + h/2
#         scaledH = h * scale
#         scaledW = w * scale

        scaledH = h * scale
        scaledW = w * scale
        lx = x
        ly = y
        
        i = 0
        
        while prediction != 1 and scale > 0.5:
            #Method 1
            rx = int(lx + scaledW)
            ry = int(ly + scaledH)
            prediction = getPrediction(grayImg[ly:ry, lx:rx])
            print(prediction)
            
            cv2.rectangle(markedImg, (lx,ly), (rx, ry), (255,0,0), 1)
            
            
            if prediction == 1:
                cv2.rectangle(markedImg, (lx,ly), (rx, ry), (0,255,0), 2)
                print("detected")
                outputImg = img.copy()
                cv2.imwrite("outputs\\" + str(counter) + ".jpg", outputImg[ly:ry, lx:rx]) 
                counter += 1
            else:
                #Method 2
                lx = int(cx - scaledW//2)
                ly = int(cy - scaledH//2)
                rx = int(cx + scaledW//2)
                ry = int(cy + scaledH//2)


                prediction = getPrediction(grayImg[ly:ry, lx:rx])
                print(prediction)

                cv2.rectangle(markedImg, (lx,ly), (rx, ry), (255,0,0), 1)


                if prediction == 1:
                    cv2.rectangle(markedImg, (lx,ly), (rx, ry), (0,255,0), 2)
                    print("detected")
                    outputImg = img.copy()
                    cv2.imwrite("outputs\\" + str(counter) + ".jpg", outputImg[ly:ry, lx:rx]) 
                    counter += 1
            
            scale = scale - 0.2
            scaledH = h * scale
            scaledW = w * scale
            print(scale)
            i+=1
            
            if i > 150:
                break

    return markedImg, counter

In [8]:
f = easygui.fileopenbox()
img = cv2.imread(f)
outputImg, facesCount = getEllipses(img)

cv2.imshow("img", outputImg)
cv2.waitKey(0)

NameError: name 'getPrediction' is not defined